In [1]:
import os

os.chdir("../")
from dotenv import load_dotenv
from knowledge_graph_creator.orchestrator import PDFToKnowledgeGraphOrchestrator

load_dotenv()

True

In [2]:
orchestrator = PDFToKnowledgeGraphOrchestrator(
    neo4j_uri=os.getenv("NEO4J_URI"),
    neo4j_user=os.getenv("NEO4J_USER"),
    neo4j_password=os.getenv("NEO4J_PASSWORD"),
    ss_api_key=os.getenv("SS_API_KEY"),
)

In [ ]:
# Process PDF and build knowledge graph
successful, unsuccessful = orchestrator.process_title_to_graph_with_network(
    parent_paper_title="Pre-train, Prompt, and Predict: A Systematic Survey of Prompting Methods in Natural Language Processing",
    citation_network_type="citations",
    include_citations=True,
    max_citations_per_paper=100,
    rate_limit_delay=0.5,
    publication_year="2021:2025",

)

2026-01-19 14:58:45.703 | INFO     | knowledge_graph_creator.orchestrator:process_title_to_graph_with_network:232 - Parent paper Details: 28692beece311a90f5fa1ca2ec9d0c2ce293d069 - Pre-train, Prompt, and Predict: A Systematic Survey of Prompting Methods in Natural Language Processing - Primary Author -Pengfei Liu
2026-01-19 14:58:48.610 | INFO     | knowledge_graph_creator.orchestrator:get_parper_to_process:177 - Number of Citation collections: 100


Adding parent paper: Pre-train, Prompt, and Predict: A Systematic Survey of Prompting Methods in Natural Language Processing


2026-01-19 14:58:49.845 | INFO     | knowledge_graph_creator.db_neo4j.academic_graph:add_paper_from_json:69 - Successfully added paper: 28692beece311a90f5fa1ca2ec9d0c2ce293d069



Processing 100 references...


Adding references:   0%|          | 0/100 [00:00<?, ?it/s]2026-01-19 14:58:51.355 | INFO     | knowledge_graph_creator.db_neo4j.academic_graph:add_paper_from_json:69 - Successfully added paper: 85bdbc26b7603f0dc6889b0042ab60e27067fd4e

Citations for 85bdbc26: 0it [00:00, ?it/s]
Adding references:   1%|          | 1/100 [00:05<09:03,  5.49s/it]2026-01-19 14:58:58.327 | INFO     | knowledge_graph_creator.db_neo4j.academic_graph:add_paper_from_json:69 - Successfully added paper: c7ec419c6ac8a7d27b7398f356acf9c01b1bafab

Citations for c7ec419c: 0it [00:00, ?it/s]
Adding references:   2%|▏         | 2/100 [00:10<08:33,  5.24s/it]2026-01-19 14:59:01.166 | INFO     | knowledge_graph_creator.db_neo4j.academic_graph:add_paper_from_json:69 - Successfully added paper: d18c98354234ad72dc13c5ec9ed8abdd9200f33a

Citations for d18c9835: 0it [00:00, ?it/s]
Adding references:   3%|▎         | 3/100 [00:13<06:33,  4.06s/it]2026-01-19 14:59:03.290 | INFO     | knowledge_graph_creator.db_neo4j.academic_gr

In [ ]:
from knowledge_graph_creator.extractors.paper_relation_extractor import PaperRelationExtractor
from knowledge_graph_creator.llm import LLMConfig, GroqModel, LLMInference

# Initialize LLM client
api_key = os.getenv("GROP_API_KEY_GRAPH")
llm_config = LLMConfig(model=GroqModel.LLAMA_8B, temperature=0.3)
llm_client = LLMInference(api_key=api_key, config=llm_config)

# Initialize extractor
extractor = PaperRelationExtractor(
    uri=os.getenv("NEO4J_URI"),
    user=os.getenv("NEO4J_USER"),
    password=os.getenv("NEO4J_PASSWORD"),
    llm_client=llm_client,
    min_delay=1
)

triplets = extractor.get_non_processed_triplets(min_citation_count=0, head_min_year=2021, tail_min_year=2025)
print(f"Found {len(triplets)} triplets")  #1709
if triplets:
    print("Sample triplet:", triplets[0])

full_results = extractor.process_all_triplets(min_citation_count=2, head_min_year=2021, tail_min_year=2024)
print(f"Processed {len(full_results)} triplets")

Found 1709 triplets
Sample triplet: {'tail_id': 'b7f9c25c565bf0f7be88d1bb0ccfbad2a61dc018', 'tail_title': 'Enhancing LLM-Based Neural Network Generation: Few-Shot Prompting and Efficient Validation for Automated Architecture Design', 'tail_abstract': 'Automated neural network architecture design remains a significant challenge in computer vision. Task diversity and computational constraints require both effective architectures and efficient search methods. Large Language Models (LLMs) present a promising alternative to computationally intensive Neural Architecture Search (NAS), but their application to architecture generation in computer vision has not been systematically studied, particularly regarding prompt engineering and validation strategies. Building on the task-agnostic NNGPT/LEMUR framework, this work introduces and validates two key contributions for computer vision. First, we present Few-Shot Architecture Prompting (FSAP), the first systematic study of the number of supporti

2026-01-19 00:46:17.239 | INFO     | knowledge_graph_creator.extractors.paper_relation_extractor:process_all_triplets:152 - Found 2548 triplets to process
2026-01-19 00:46:17.243 | INFO     | knowledge_graph_creator.extractors.paper_relation_extractor:process_all_triplets:165 - Processing 1/2548: b7f9c25c565bf0f7be88d1bb0ccfbad2a61dc018 -> acbdbf49f9bc3f151b93d9ca9a06009f4f6eb269
2026-01-19 00:46:20.845 | INFO     | knowledge_graph_creator.extractors.paper_relation_extractor:process_all_triplets:165 - Processing 2/2548: 07b408f9914248d1e09f6e7d74bc0f049f4e73ea -> acbdbf49f9bc3f151b93d9ca9a06009f4f6eb269
2026-01-19 00:46:21.915 | ERROR    | knowledge_graph_creator.extractors.paper_relation_extractor:extract_relation_with_structured_llm:100 - LLM extraction failed: Error code: 400 - {'error': {'message': "Failed to call a function. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'tool_use_failed', 'failed_generation': '<func